In [1]:
import pandas as pd
import os
from datetime import datetime
import utils.data_utils as data_utils
from collections import defaultdict

LONG_TERM_ACTIONS = ['walking', 'eating', 'smoking', 'discussion']
ACTIONS = data_utils.define_actions('all')
ACTIONS.sort(key=len)

In [2]:
def csv_iter():
    for f in os.listdir('./checkpoint/test'):
        try:
            datetime.strptime(f[-23:-4], "%d-%m-%Y-%H:%M:%S")
            if f.split('.')[-1] == 'csv':
                day = int(f[-23:-21])
                if day > 10:
                    yield f
        except:
            pass

def extend_df(df, long_term=True):
    if long_term:
        actions = LONG_TERM_ACTIONS
        time_lens = [560, 1000]
    else:
        actions = ACTIONS
        time_lens = [80, 160, 320, 400]
    columns_mapping = defaultdict(list)
    for time_len in time_lens:
        for action_name in actions:
            columns_mapping['3d' + str(time_len)].append(action_name + '3d' + str(time_len))
            columns_mapping['usingfulltestset_3d' + str(time_len)].append('usingfulltestset_' + action_name + '3d' + str(time_len))

    for key in columns_mapping:
        df[key] = df[columns_mapping[key]].mean(axis=1)
    
    return list(columns_mapping.keys())

def get_summary_df(merged):
    cols = list(set([i[:-5] for i in merged.columns]))
    new_cols = []
    for col in cols:
        col_name = [c for c in merged.columns if c.startswith(col)]
        new_cols.append(col+'_mean')
        new_cols.append(col+'_std')
        merged[col+'_mean'] = merged[col_name].mean(axis=1)
        merged[col+'_std'] = merged[col_name].std(axis=1)
    return merged[new_cols]

In [4]:
df_lst = []
mini_df_lst = []
for i, f in enumerate(csv_iter()):
    if not f.startswith('4-4-6-6-8-8-10-10'):
        continue
    print(i, f)
    df = pd.read_csv(os.path.join('checkpoint/test', "./", f))
    df = df.sort_values(by='v_3d')
    df_lst.append(df[extend_df(df, LONG_TERM_ACTIONS)].add_suffix('_run' + str(i)))
    mini_df_lst.append(df[extend_df(df, LONG_TERM_ACTIONS)].head(1).add_suffix('_run' + str(i)).reset_index(drop=True))

merged = pd.concat(mini_df_lst, axis=1)

0 4-4-6-6-8-8-10-10main_3d_3D_in10_out25_dct_n_30_22-06-2020-10:33:39.csv
2 4-4-6-6-8-8-10-10main_3d_3D_in10_out25_dct_n_30_24-06-2020-10:30:18.csv
4 4-4-6-6-8-8-10-10main_3d_3D_in10_out25_dct_n_30_23-06-2020-17:57:26.csv
6 4-4-6-6-8-8-10-10main_3d_3D_in10_out25_dct_n_30_23-06-2020-02:19:14.csv
9 4-4-6-6-8-8-10-10main_3d_3D_in10_out25_dct_n_30_21-06-2020-17:04:51.csv


In [5]:
get_summary_df(merged)

,usingfulltestset_3d560_mean,usingfulltestset_3d560_std,usingfulltestset_3d1000_mean,usingfulltestset_3d1000_std,3d1000_mean,3d1000_std,3d560_mean,3d560_std
0,60.339761,0.725688,80.753044,0.70724,68.916667,0.844627,50.056121,0.435249


In [6]:
mini_df_lst[0]

,3d560_run0,usingfulltestset_3d560_run0,3d1000_run0,usingfulltestset_3d1000_run0
0,50.523773,59.612364,68.232851,80.248953


In [7]:
mini_df_lst[1]

,3d560_run2,usingfulltestset_3d560_run2,3d1000_run2,usingfulltestset_3d1000_run2
0,49.637818,60.783626,68.056699,80.915037


In [8]:
mini_df_lst[2]

,3d560_run4,usingfulltestset_3d560_run4,3d1000_run4,usingfulltestset_3d1000_run4
0,50.459402,61.079183,69.452363,81.431589


In [9]:
mini_df_lst[3]

,3d560_run6,usingfulltestset_3d560_run6,3d1000_run6,usingfulltestset_3d1000_run6
0,50.052311,60.715848,70.073143,81.360507


In [10]:
mini_df_lst[4]

,3d560_run9,usingfulltestset_3d560_run9,3d1000_run9,usingfulltestset_3d1000_run9
0,49.607301,59.507786,68.768278,79.809133
